## Problem:

"Amandalle voi laittaa viestiä osoitteeseen amanda@gmail.com"

$\rightarrow$ Amandalle ja amanda ei maskaudu oikein.
$\rightarrow$ Lemmatisointi!

In [1]:
!pip -q install spacy
! python -m spacy download fi_core_news_lg


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 MB 19.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('fi_core_news_lg')


In [2]:
import spacy
import transformers
import datasets
import torch
import random
import copy
import itertools
import numpy as np
import string
#from cosine_similarity import find_best_cosine_match

In [3]:
lem = spacy.load("fi_core_news_lg")
doc = lem("Amandalle voi laittaa viestiä osoitteeseen amanda@gmail.com. Viesti suomeksi.")
for token in doc:
    #print(token.text, token.pos_, token.dep_)
    print(token.lemma_)
    

Amanda
voida
laittaa
viesti
osoitteeseen
amanda@gmail.com
.
viesti
suomi
.


In [4]:
# how about a list?

lista = ["amandalle", "voi", "laittaa", "viestiä"]
lemmatized = [t.lemma_ for t in lem(" ".join(lista))]
print(lemmatized)

['amanda', 'voida', 'laittaa', 'viesti']


### Idea: 
lemmatize at the beginning and mask a lemmatized, lowercased sentence? And at the end find overlaps?
=> messes up witht the grammar that is essential for making predictions.

## Modification of the word-list in masking

In [17]:
class PiiMasker:

    def __init__(self, model, tokenizer, threshold, use_context=False, choose_n=100, choose_k=3, embedding_model=None, tokenizer_type="BPE", return_tokenizer_output=False):
        self.model = model
        self.model.eval()   # remove some unneeded functionality
        self.tokenizer = tokenizer
        assert tokenizer_type in ["BPE", "WordPiece"]
        self.tokenizer_type = tokenizer_type
        self.continuation_marker = {"BPE": "▁", "WordPiece": "##"}[tokenizer_type] 
        self.special_tokens = tokenizer.all_special_tokens
        self.threshold = threshold    # this is to be optimized
        self.use_context = bool(use_context)
        self.choose_n = int(choose_n)
        self.choose_k = int(choose_k)
        self.return_tokenizer_output=return_tokenizer_output
        if embedding_model is not None:
            self.embedding_model = embedding_model
        else:
            self.embedding_model = self.model

    def find_pii(self, text, debug = False):
        masked_indices, tokenized_text, decoded_text, context = self.mask(text)
        if context == []:
            context = [[]*len(masked_indices)]
        if debug: print(masked_indices)
        to_be_redacted = []
        to_be_redacted_words = []
        to_redact_with = []
        to_redact_context = []
        for ind, cont in zip(masked_indices, context):
            final_score, predictions = self.get_scores(ind, tokenized_text, cont, debug)
            word = self.tokenizer.decode(tokenized_text["input_ids"][0][ind])
            if final_score < self.threshold:
                to_be_redacted.append(ind)
                to_redact_with.append(predictions)
                to_be_redacted_words.append(self.tokenizer.convert_ids_to_tokens(tokenized_text["input_ids"][0][ind]))
                to_redact_context.append(cont)
        if self.return_tokenizer_output:
            return {"decoded_text": decoded_text, 
                    "tokenizer_output": tokenized_text, 
                    "to_redact_indices": to_be_redacted, 
                    "to_redact_words": to_be_redacted_words, 
                    "predictions": to_redact_with,
                    "context": to_redact_context}
        else:
            return {"decoded_text": decoded_text,
                    "to_redact_indices": to_be_redacted, 
                    "to_redact_words": to_be_redacted_words,
                    "predictions": to_redact_with,
                    "context": to_redact_context}

    def print_pii(self, text, debug=False):
        masked_indices, tokenized_text, decoded_text, context = self.mask(text)
        if context == []:
            context = [[]*len(masked_indices)]
        prints = []
        for ind, cont in zip(masked_indices, context):
            final_score, predictions = self.get_scores(ind, tokenized_text, cont, debug)
            word = self.tokenizer.decode(tokenized_text["input_ids"][0][ind])
            if final_score < self.threshold:
                prints.append(f'{word} \t >> {final_score} \t >> Redact')#, preds: {predictions[:4]}')
            else:
                prints.append(f'{word} \t >> {final_score}')
        for p in prints:
            print(p)


    
#---------------------------------MASKING---------------------------------------#
    def get_indices_WordPiece(self, t):
        converted = self.tokenizer.convert_ids_to_tokens(t["input_ids"][0])
        indices=[]
        for i in range(0, len(t["input_ids"][0])):
            if converted[i][:2] != self.continuation_marker and converted[i] not in self.special_tokens:
                indices.append([i])
            else:
                if converted[i] not in self.special_tokens and indices!=[]:   # here we are only skipping the fact that first token is a special token; indices is empty.
                    indices[-1].append(i)
        return indices

    def get_indices_BPE(self, t):
        converted = self.tokenizer.convert_ids_to_tokens(t["input_ids"][0])
        indices=[]
        reminder = False   # for BPE, to separate punct, we need to know if the last token was punct
        for i in range(0, len(t["input_ids"][0])):
            # for BPE, continuation marker is actually a "starting marker"
            if reminder or (converted[i][0] == self.continuation_marker and converted[i] not in self.special_tokens) or converted[i] in string.punctuation:
                reminder=False
                if converted[i] in string.punctuation:
                    reminder = True
                indices.append([i])
            else:
                if converted[i] not in self.special_tokens and indices!=[]:   
                    # here we are only skipping the fact that first token is a special token; indices is empty.
                    indices[-1].append(i)
        return indices

    def mask(self, text):
        if self.use_context:
            return self.context_aware_mask(text)
        t = self.tokenizer(text, return_tensors='pt') # prepare normal tokenized input
        if self.tokenizer_type == "BPE":
            indices = self.get_indices_BPE(t)
        elif self.tokenizer_type == "WordPiece":
            indices = self.get_indices_WordPiece(t)
        return indices, t, self.tokenizer.decode(t.input_ids[0]), [[]]*len(indices)   # []s for empty context
        
#----------------------------CONTEXT AWARE MASKING---------------------------------#
    def find_same_tokens(self, lst, index):
        target_value = lst[index]
        return [i for i, value in enumerate(lst) if value == target_value and i != index]
    
    def get_context_indices_WordPiece(self, t):
        converted = self.tokenizer.convert_ids_to_tokens(t["input_ids"][0])
        indices=[]
        words = []
    
        # first, getting the indices as above, but also saving the words in lowercase
        for i in range(0, len(t["input_ids"][0])):
            if converted[i][:2] != self.continuation_marker and converted[i] not in self.special_tokens:
                indices.append([i])
                words.append(converted[i].lower())
            else:
                if converted[i] not in self.special_tokens and indices!=[]:   
                    # here we are only skipping the fact that first token is a special token; indices is empty.
                    indices[-1].append(i)
                    words[-1] += converted[i][2:].lower()
        
        indices_context=[]
        lem_words = [t.lemma_ for t in lem(" ".join(words))]
        assert len(lem_words)==len(indices), "Issues with masking the sentence."
    
        # then, map same words to same context: eg. "Ville" in indices [2,3] and [13,14]
        # => context for first is [13,14] and [2,3] for second.
        for i in range(len(lem_words)):
            ind_of_words = self.find_same_tokens(lem_words, i)
            if ind_of_words != []:
                print(words[i],":", ind_of_words, np.array(words)[ind_of_words])
                current = []
                for j in ind_of_words:
                    current+= indices[j]
                indices_context.append(current)
            else:
                indices_context.append([])
        
        assert len(indices)==len(indices_context), "Issues with context masking, "+str(len(indices))+"!="+str(len(indices_context))+"\nIndices:\t"+str(indices)+"\nContext:\t"+str(indices_context)
        return indices, indices_context 
    
    def get_context_indices_BPE(self, t):
        converted = self.tokenizer.convert_ids_to_tokens(t["input_ids"][0])
        indices=[]
        words = []
    
        reminder = False   # for BPE, to separate punct, we need to know if the last token was punct
        for i in range(0, len(t["input_ids"][0])):
            # for BPE, continuation marker is actually a "starting marker"
            if reminder or (converted[i][0] == self.continuation_marker and converted[i] not in self.special_tokens) or converted[i] in string.punctuation:
                reminder=False
                if converted[i] in string.punctuation:
                    reminder = True
                indices.append([i])
                if converted[i][0] == self.continuation_marker:
                    words.append(converted[i][1:].lower())
                else:
                    words.append(converted[i].lower())
            else:
                if converted[i] not in self.special_tokens and indices!=[]:   
                    # here we are only skipping the fact that first token is a special token; indices is empty.
                    indices[-1].append(i)
                    words[-1] += converted[i].lower()
    
        indices_context=[]
        lem_words = [t.lemma_ for t in lem(" ".join(words))]
        assert len(lem_words)==len(indices), "Issues with masking the sentence."
    
        # then, map same words to same context: eg. "Ville" in indices [2,3] and [13,14]
        # => context for first is [13,14] and [2,3] for second.
        for i in range(len(words)):
            ind_of_words = self.find_same_tokens(lem_words, i)
            if ind_of_words != []:
                print(words[i],":", ind_of_words, np.array(words)[ind_of_words])
                current = []
                for j in ind_of_words:
                    current+= indices[j]
                indices_context.append(current)
            else:
                indices_context.append([])
        
        assert len(indices)==len(indices_context), "Issues with context masking, "+str(len(indices))+"!="+str(len(indices_context))+"\nIndices:\t"+str(indices)+"\nContext:\t"+str(indices_context)
        return indices, indices_context 
    
    
    def context_aware_mask(self, text):
        
        t = self.tokenizer(text, return_tensors='pt') # prepare normal tokenized input
        if self.tokenizer_type == "BPE":
            indices, context = self.get_context_indices_BPE(t)
        elif self.tokenizer_type == "WordPiece":
            indices,  context = self.get_context_indices_WordPiece(t)
    
        return indices, t, self.tokenizer.decode(t.input_ids[0]), context


#-------------------------------predictions-------------------------------------#

    def predict(self, masked, i, true_token, print_results=False):

        def to_probability(A):
            softmax = torch.nn.Softmax(dim=0)
            return softmax(A)
            
        # do a prediction
        with torch.no_grad():
            model_out = self.model(**masked)
        if self.tokenizer_type=="WordPiece":
            logits = model_out["prediction_logits"]
        elif self.tokenizer_type=="BPE":
            logits = model_out["logits"]
    
        # logits for this word specifically
        logits_i = logits[0,i,:]  # this contains the probabilities for this token
        # change to probability
        probs = to_probability(logits_i)
        # true token is the index
        word_probability = probs[true_token]

        top_logits, top_tokens = torch.sort(logits, dim=2, descending=True)#[:,:,:self.choose_n]
        top_tokens = top_tokens[:,:,:self.choose_n]
        top_guesses = [self.tokenizer.decode(g) for g in top_tokens[0,i,:]]
        #if print_results: print(f'{self.tokenizer.decode(true_token)} has best guesses {top_guess} and probability {word_probability}')

        
        # Do only in debug mode:
        if print_results:
            print(f'{self.tokenizer.decode(true_token)} has probability {word_probability}')
            # see choose_n predictions for debug
            top_probs = to_probability(top_logits[0,i,:])
            top_logits = top_logits[:,:,:self.choose_n]
            top_tokens = top_tokens[:,:,:self.choose_n]
    
        
            #print("Guesses:",self.tokenizer.decode(top_tokens[0,i,:]))
            #print("Logits: ",top_logits[0,i,:])
            #print("Probs:  ",top_probs[:self.choose_n])
            print("")
        return word_probability, top_guesses

    def get_scores(self, to_be_masked, tokens, context, debug):
        """
        Calculates the (aggregated) probability of the given word based on the model prediction.
        For multi-subtoken words, aggregation strategy is gradual unmasking and multiplication.
        Input: 
            tokens: tokenizer output for a span of text
            to_be_masked: indices for which are masked from the tokens and over which we calculate
                          i.e. indices of the subtokens that form a word.
            debug (False): prints out extra information if True
        Returns:
            (aggregated) probability \in (0,1)
        """
        # initialize the score; we're multiplying, so 1
        final_score = 1
        predictions = []
    
        # loop over the subtokens of a word
        for i in range(len(to_be_masked)):
            # making a deep copy as tensors are nested and yada yada
            t = copy.deepcopy(tokens)
            current = to_be_masked[i:]   # this is the token we are CURRENTLY interested in
            for j in current:
                t["input_ids"][0][j] = self.tokenizer.mask_token_id     # we mask the SUBtokens that are in current
            if context != []:   # if we have context, mask that
                for j in context:
                    t["input_ids"][0][j] = self.tokenizer.mask_token_id 
            if debug:
                print(self.tokenizer.decode(t["input_ids"][0]))
            # multiply the final score with the predicted probability => aggregates over to_be_masked==one word
            score, preds = self.predict(t, current[0], tokens.input_ids[0][current[0]], print_results=debug)
            final_score *= score
            predictions.append(preds)
            
        return final_score, predictions

    

In [6]:
import transformers

In [18]:
MODEL_NAME="TurkuNLP/bert-base-finnish-cased-v1"
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
model = transformers.AutoModelForPreTraining.from_pretrained(MODEL_NAME)

pf = PiiMasker(model, tokenizer, 1e-4, use_context=True, tokenizer_type="WordPiece")

In [20]:
pf.find_pii("Moi, olen Amanda. Amandalle voit laittaa viestiä osoitteeseen amanda@gmail.com. Viesti suomeksi.", debug=True)

amanda : [5, 10] ['amandalle' 'amanda']
. : [13, 15, 18] ['.' '.' '.']
amandalle : [3, 10] ['amanda' 'amanda']
viestiä : [16] ['viesti']
amanda : [3, 5] ['amanda' 'amandalle']
. : [4, 15, 18] ['.' '.' '.']
. : [4, 13, 18] ['.' '.' '.']
viesti : [8] ['viestiä']
. : [4, 13, 15] ['.' '.' '.']
[[1], [2], [3], [4, 5], [6], [7, 8], [9], [10], [11], [12], [13, 14, 15], [16], [17], [18], [19], [20], [21], [22], [23]]
[CLS] [MASK], olen Amanda. Amandalle voit laittaa viestiä osoitteeseen amanda @ gmail. com. Viesti suomeksi. [SEP]
Moi has probability 0.07974661141633987

[CLS] Moi [MASK] olen Amanda. Amandalle voit laittaa viestiä osoitteeseen amanda @ gmail. com. Viesti suomeksi. [SEP]
, has probability 0.927946150302887

[CLS] Moi, [MASK] Amanda. Amandalle voit laittaa viestiä osoitteeseen amanda @ gmail. com. Viesti suomeksi. [SEP]
olen has probability 0.3216909170150757

[CLS] Moi, olen [MASK] [MASK]. [MASK] [MASK] voit laittaa viestiä osoitteeseen [MASK] [MASK] [MASK] @ gmail. com. Viesti 

{'decoded_text': '[CLS] Moi, olen Amanda. Amandalle voit laittaa viestiä osoitteeseen amanda @ gmail. com. Viesti suomeksi. [SEP]',
 'to_redact_indices': [[4, 5], [7, 8], [13, 14, 15]],
 'to_redact_words': [['Aman', '##da'],
  ['Aman', '##dalle'],
  ['am', '##and', '##a']],
 'predictions': [[['kiinnostunut',
    'nuori',
    '24',
    'myös',
    'siis',
    '17',
    'tyttö',
    'itse',
    '23',
    '22',
    '16',
    '18',
    '25',
    'nainen',
    '19',
    'naispuol',
    'ihan',
    '20',
    'jo',
    '26',
    '21',
    'kirjoittanut',
    'lukenut',
    'täällä',
    '15',
    '14',
    '27',
    'juuri',
    'vasta',
    'suomalainen',
    'uusi',
    'muuttamassa',
    'muuttanut',
    'suomenruotsa',
    '13',
    'Jenni',
    'kotoisin',
    '28',
    'todella',
    'tosi',
    'lähdössä',
    'aika',
    '31',
    '.',
    'itsekin',
    'nyt',
    'aivan',
    '29',
    'asiakaspalvelu',
    'tamper',
    '30',
    'ja',
    'ihastunut',
    'käynyt',
    'nais',
   